In [14]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import time
import datetime as dt

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [18]:
# Toxic, Promiscuous, Sk8er Boi, Run It!, Your Love Is My Drug
track_uris = ["spotify:track:6I9VzXrHxO9rA9A5euc8Ak", "spotify:track:2gam98EZKrF9XuOkU13ApN", "spotify:track:4omisSlTk6Dsq2iQD7MA07", "spotify:track:7xYnUQigPoIDAMPVK79NEq", "spotify:track:3uoQULcUWfnt6nc6J7Vgai"]

In [52]:
# all_features = pd.DataFrame(sp.audio_features(track_uris))
chosen_features = sp.audio_features(track_uris)
# print(all_features)

In [53]:
recommendations_uris = pd.DataFrame(sp.recommendations(seed_tracks=track_uris, limit=100)['tracks'])['uri']
# recomendations_features = pd.DataFrame(sp.audio_features(recommendations_uris))
recomendations_features = sp.audio_features(recommendations_uris)
# print(recomendations_features)

In [56]:
all_features = pd.DataFrame(chosen_features + recomendations_features)
# print(all_features)

In [57]:
usable_features = all_features.drop(['key', 'loudness', 'mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis=1)
usable_features['tempo'] = usable_features['tempo'] / 200
print(usable_features)

     danceability  energy  speechiness  acousticness  instrumentalness  \
0           0.774   0.838       0.1140      0.024900          0.025000   
1           0.808   0.970       0.0506      0.056900          0.000061   
2           0.487   0.900       0.0482      0.000068          0.000000   
3           0.846   0.482       0.1290      0.024600          0.000000   
4           0.829   0.627       0.0759      0.006630          0.000000   
..            ...     ...          ...           ...               ...   
100         0.709   0.745       0.0738      0.022500          0.000052   
101         0.465   0.956       0.1280      0.009630          0.000101   
102         0.731   0.807       0.1000      0.181000          0.751000   
103         0.541   0.953       0.1070      0.048000          0.000000   
104         0.832   0.391       0.0628      0.056400          0.000000   

     liveness  valence     tempo  
0      0.2420    0.924  0.715200  
1      0.1540    0.868  0.571640  
2     

In [58]:
import scipy.stats as stats

In [59]:
stats.gaussian_kde(all_features)

TypeError: No loop matching the specified signature and casting was found for ufunc add